In [ ]:
adata = sc.read_h5ad(
    "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.1_cellbin_rawdata/filtered_data/merged_adata2.h5ad"
)
adata_tmp = sc.read(
    f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/SPACEL/Spoint_annotation/SS200000118TL_C1/Spiont_SS200000118TL_C1.h5ad"
)

In [ ]:
celltype_score = adata.obs[adata_tmp.uns["celltypes"].tolist() + ["celltype", "cellsubtype"]].copy()
celltype_score[adata_tmp.uns["celltypes"].tolist()] = celltype_score[adata_tmp.uns["celltypes"].tolist()].fillna(0)
celltype_score = celltype_score[~celltype_score["celltype"].isin(["Nerve_Cell"])]
celltype_score = celltype_score[~celltype_score["cellsubtype"].isin(["Acinar_Cell"])]

In [ ]:
# make a dict for celltype as key and cellsubtypes as value list
celltype_subtype_dict = {}
for celltype in celltype_score["celltype"].unique():
    celltype_subtype_dict[celltype] = (
        celltype_score[celltype_score["celltype"] == celltype]["cellsubtype"].unique().tolist()
    )
celltype_subtype_dict
# group celltype_score.columns by celltype_subtype_dict and get sum value
celltype_score_sum = pd.DataFrame()
for key, value in celltype_subtype_dict.items():
    celltype_score_sum[key] = celltype_score[value].sum(axis=1)
# celltype_score_sum['celltype'] = celltype_score['celltype']
celltype_score_sum["celltype"] = celltype_score["celltype"]
celltype_score_sum["celltype"] = celltype_score_sum["celltype"].cat.remove_unused_categories()

In [ ]:
# plot heatmap for celltype_score_sum.groupby('celltype').mean()
celltype_score_sum_mean = celltype_score_sum.groupby("celltype").mean()
# sort columns and index of celltype_score_sum_mean
celltype_score_sum_mean = celltype_score_sum_mean.reindex(sorted(celltype_score_sum_mean.columns), axis=1)
import seaborn as sns

g = sns.clustermap(
    celltype_score_sum_mean,
    z_score=1,
    row_cluster=False,
    col_cluster=False,
    cmap="RdBu_r",
    linewidth=0.3,
    linecolor="#929292",
    figsize=(5.5, 5),
)
g.ax_cbar.set_position((0.9, 0.1, 0.03, 0.2))
g.ax_cbar.set_title("Score")
g.ax_cbar.tick_params(axis="x", length=1)
for spine in g.ax_cbar.spines:
    g.ax_cbar.spines[spine].set_color("#929292")
    g.ax_cbar.spines[spine].set_linewidth(1)
plt.savefig("celltype_score_heatmap.pdf", dpi=300, bbox_inches="tight")

In [ ]:
import os, sys
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from STutils.pl import plot_cellbin_gradient, plot_cellbin_discrete, getDefaultColors

In [ ]:
sc.settings.set_figure_params(
    dpi=100, dpi_save=300, frameon=False, facecolor="white", fontsize=16, vector_friendly=True, figsize=(5, 5)
)
sc._settings.ScanpyConfig(figdir="./", n_jobs=30)

In [ ]:
od = "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.2_cellbin_spatial_plot/celltype_spatial_plot3"
os.system(f"mkdir -p {od}")
os.chdir(od)

In [ ]:
sample = "SS200000148TR_D1"

In [ ]:
adata = sc.read(
    f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.1_cellbin_rawdata/rawdata/adata_{sample_dict[sample]}.h5ad"
)

In [ ]:
rename_dict = {
    "Fibroblast_Cell": "Fibroblast",
    "Acinar_Cell": "Acinar",
    "Endothelial_Cell": "Endothelial",
    "PVL_Cell": "Stellate",
    "Endocrine_Cell": "Endocrine",
    "Nerve_Cell": "Neuron",
    "Macrophage_Cell": "Macrophage",
}
adata.obs["celltype"] = adata.obs["celltype"].replace(rename_dict)
adata.obs["celltype"] = pd.Categorical(adata.obs["celltype"], categories=sorted(adata.obs["celltype"].unique()))

In [ ]:
resolution = "celltype"
clusters = adata.obs[resolution].cat.categories
cluster_number = clusters.shape[0]
colors = getDefaultColors(cluster_number, type=2)
flout = open(f"color_{resolution}_{sample_dict[sample]}.list", "w")
for i in range(cluster_number):
    if clusters[i] == "Low_Quanlity_Cell":
        flout.write(clusters[i] + "\t#808080\n")
    elif clusters[i] == "Undefined_Cell":
        flout.write(clusters[i] + "\t#D3D3D3\n")
    else:
        flout.write(clusters[i] + "\t" + colors[i] + "\n")
flout.close()

In [ ]:
from collections import OrderedDict
from PIL import Image

Image.MAX_IMAGE_PIXELS = None


def plot_cellbin_discrete(
    adata, mask, tag, prefix, colors, dpi=600, edge_cut=300, add_scale_bar=True, scale=0.5, length_fraction=0.25
):
    """
    params:
        adata: AnnData object
        mask: mask file path
        tag: tag name in adata.obs
        prefix: prefix of output file
        colors: color palette in getDefaultcolors
        dpi: dpi of output file
        edge_cut: pixels to retain in edge cutting
    """
    res = pd.DataFrame(adata.obs, columns=["x", "y", tag], index=adata.obs.index)
    res = res.sort_values(by=tag)
    res.to_csv(f"bin1clu_{tag}_{prefix}.txt", sep="\t", index=False)  # write to file
    clusters = res[tag].unique()
    cluster_number = clusters.shape[0]
    color_palette_fl = open(colors, "r").readlines()
    color_palette = OrderedDict()
    for line in color_palette_fl:
        color_palette[line.split("\t")[0]] = line.split("\t")[1].strip()
    # sort clusters by color_palette.keys()
    clusters = [cluster for cluster in color_palette.keys() if cluster in clusters]
    # colors = getDefaultColors(cluster_number, type=colors)
    # flout = open(f"color_{tag}_{prefix}.list", 'w')
    # for i in range(cluster_number):
    #     flout.write(str(clusters[i]) + '\t' + colors[i] + '\n')
    # flout.close()  # write to file
    tif = f"{tag}_{prefix}_spatial.tif"
    os.system(
        f"/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/bin/cell_bin_plot bin1clu_{tag}_{prefix}.txt {mask} {colors} {tif}"
    )  # draw cellbin discrete tif
    img = plt.imread(tif)
    im = img.copy()
    # cut black edge of im
    non_black_coords = np.argwhere(im.sum(axis=2) > 0)
    y1, x1 = non_black_coords.min(axis=0)
    y2, x2 = non_black_coords.max(axis=0)
    im = im[y1 - edge_cut : y2 + 1000, x1 - edge_cut : x2 + edge_cut]
    # replace black background with white
    black_pixels = (im[:, :, 0] == 0) & (im[:, :, 1] == 0) & (im[:, :, 2] == 0)
    im[black_pixels] = [255, 255, 255]
    # draw color legend
    fig, ax = plt.subplots(figsize=(5, 5), dpi=dpi, gridspec_kw={"wspace": 0, "hspace": 0})
    plt.subplots_adjust(0, 0, 1, 1)
    ax.axis("off")
    ax.imshow(im)
    if add_scale_bar:
        from matplotlib_scalebar.scalebar import ScaleBar

        scalebar = ScaleBar(scale, "um", length_fraction=length_fraction, frameon=False, location="lower right")
        ax.add_artist(scalebar)
    # fig.patch.set_facecolor('black')
    # Create a legend for the discrete colors
    legend_labels = [str(cluster) for cluster in clusters]  # Assuming clusters is a list of labels
    legend_patches = [
        mpl.patches.Patch(color=color_palette[legend_labels[i]], label=legend_labels[i]) for i in range(cluster_number)
    ]

    # Add a legend to the figure
    legend = ax.legend(handles=legend_patches, loc="lower left", fontsize=8, bbox_to_anchor=(1.05, 0))
    legend.set_title(tag, prop={"size": 8, "weight": "bold"})
    # set legend text color white
    legend.get_title().set_color("black")
    for text in legend.get_texts():
        text.set_color("black")
    outfig = f"{tag}_{prefix}_spatial.pdf"
    fig.savefig(outfig, dpi=dpi, format="pdf", bbox_inches="tight")
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5), dpi=300, gridspec_kw={"wspace": 0, "hspace": 0})
ax.axis("off")
colors = "/jdfsbjcas1/ST_BJ/P21H28400N0232/gongchanghao/project/PDAC/spatial/cellbin_v3/1.2_cellbin_spatial_plot/celltype_spatial_plot3/color_celltype.list"
color_palette_fl = open(colors, "r").readlines()
color_palette = {}
for line in color_palette_fl:
    color_palette[line.split("\t")[0]] = line.split("\t")[1].strip()
clusters = list(color_palette.keys())
legend_labels = [str(cluster) for cluster in clusters]  # Assuming clusters is a list of labels
legend_patches = [
    mpl.patches.Patch(color=color_palette[legend_labels[i]], label=legend_labels[i]) for i in range(len(legend_labels))
]

# Add a legend to the figure
legend = ax.legend(handles=legend_patches, loc="lower left", fontsize=8, bbox_to_anchor=(1.05, 0), ncol=8)
legend.set_title("CellType", prop={"size": 12, "weight": "bold"})
# set legend text color white
legend.get_title().set_color("black")
for text in legend.get_texts():
    text.set_color("black")
plt.savefig("legend.pdf", dpi=300, format="pdf", bbox_inches="tight")